# FLEKS Python Visualization Toolkit: AMReX Data

flekspy is a Python package for processing FLEKS data. This notebook focuses on handling data in the AMReX format.

## FLEKS data format

* Field: *.out format or AMREX built-in format, whose directory name is assumed to end with "_amrex"
* PIC particle: AMREX built-in format 
* Test particle: binary data format

## Importing the package

In [ ]:
import flekspy

## Downloading demo data

Example PIC particle data can be downloaded as follows:

In [ ]:
from flekspy.util import download_testfile

url = "https://raw.githubusercontent.com/henry2004y/batsrus_data/master/3d_particle.tar.gz"
download_testfile(url, "data")

Native AMReX format can be read using YT:

In [ ]:
ds = flekspy.load("data/3d*amrex")

We can take a slice at a given location, which works both for IDL and AMReX data:

In [ ]:
dc = ds.get_slice("z", 0.5)
dc

and plot the selected variables on the slice with colored contours, streamlines, and contour lines:

In [ ]:
f, axes = dc.plot("By", pcolor=True)
dc.add_stream(axes[0], "Bx", "By", color="w")
dc.add_contour(axes[0], "Bx")

Users can also obtain the data of a 2D plane, and visualize it with matplotlib. A complete example is given below:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

ds = flekspy.load("data/3*amrex")
dc = ds.get_slice("z", 0.5)

f, axes = plt.subplots(
    1, 2, figsize=(12, 4), constrained_layout=True, sharex=True, sharey=True
)

fields = ["By", "Bz"]
for ivar in range(2):
    v = dc.evaluate_expression(fields[ivar])
    vmin = v.min().v
    vmax = v.max().v
    ax = axes[ivar]
    ax.set_title(fields[ivar], fontsize=16)
    ax.set_ylabel("Y", fontsize=16)
    ax.set_xlabel("X", fontsize=16)
    c = ax.pcolormesh(dc.x.value, dc.y.value, np.array(v.T), cmap="turbo")
    cb = f.colorbar(c, ax=ax, pad=0.01)

    ax.set_xlim(np.min(dc.x.value), np.max(dc.x.value))
    ax.set_xlim(np.min(dc.y.value), np.max(dc.y.value))
    dc.add_stream(ax, "Bx", "By", density=0.5, color="w")

plt.show()

### Experimental AMReX Particle Loader

After flekspy v0.5.23, we introduced an experimental native AMReX particle loader.

In [ ]:
from flekspy.amrex import AMReXParticleData

data_file = "data/3d_particle_region0_1_t00000002_n00000007_amrex"
pd = AMReXParticleData(data_file)
pd

Particles within a given region can be extracted as follows:

In [ ]:
x_range = (-0.008, 0.008)
y_range = (-0.005, 0.005)

rdata = pd.select_particles_in_region(x_range=x_range, y_range=y_range)

Phase space plotting is achieved via `plot_phase`:

In [ ]:
fig, ax = pd.plot_phase("velocity_y", "velocity_z", bins=(100,32))
plt.show()

## Visualizing phase space distributions with yt

In [ ]:
from flekspy.util import unit_one

data_file = "data/3d_*amrex"
ds = flekspy.load(data_file)

# Add a user defined field. See yt document for more information about derived field.
ds.add_field(
    ds.pvar("unit_one"),
    function=unit_one,
    sampling_type="particle",
    units="dimensionless",
)

x_field = "p_uy"
y_field = "p_uz"
# z_field = "unit_one"
z_field = "p_w"
xleft = [-0.016, -0.01, 0.0]
xright = [0.016, 0.01, 1.0]

zmin, zmax = 1e-5, 2e-3

## Select and plot the particles inside a box defined by xleft and xright
region = ds.box(xleft, xright)
pp = ds.plot_phase(
    x_field,
    y_field,
    z_field,
    region=region,
    unit_type="si",
    x_bins=100,
    y_bins=32,
    domain_size=(xleft[0], xright[0], xleft[1], xright[1]),
)

pp.set_cmap(pp.fields[0], "turbo")
pp.set_font(
    {
        "size": 34,
        "family": "DejaVu Sans",
    }
)

pp.set_zlim(pp.fields[0], zmin=zmin, zmax=zmax)

pp.set_xlabel(r"$V_y$")
pp.set_ylabel(r"$V_z$")
pp.set_colorbar_label(pp.fields[0], "weight")
str_title = (
    rf"x = [{xleft[0]:.1e}, {xright[0]:.1e}], y = [{xleft[1]:.1e}, {xright[1]:.1e}]"
)
pp.set_title(pp.fields[0], str_title)
pp.set_log(pp.fields[0], True)
pp.show()
# pp.save("test")
# pp.plots[("particles", z_field)].axes.xaxis.set_major_locator(
# ticker.MaxNLocator(4))
# pp.plots[("particles", z_field)].axes.yaxis.set_major_locator(
# ticker.MaxNLocator(4))

### Plot the location of particles that are inside a sphere

In [ ]:
center = [0, 0, 0]
radius = 1
# Object sphere is defined in yt/data_objects/selection_objects/spheroids.py
sp = ds.sphere(center, radius)
pp = ds.plot_particles(
    "p_x", "p_y", "p_w", region=sp, unit_type="si", x_bins=64, y_bins=64
)
pp.show()
# pp.save("test")

### Plot the velocity space of particles that are inside a sphere

In [ ]:
pp = ds.plot_phase(
    "p_uy", "p_uz", "p_w", region=sp, unit_type="si", x_bins=64, y_bins=64
)

pp.set_cmap(pp.fields[0], "turbo")

pp.set_font(

    {

        "size": 34,

        "family": "DejaVu Sans",

    }
)

pp.show()

### Plot the location of particles that are inside a disk

In [ ]:
center = [0, 0, 0]
normal = [0, 0, 1]  # normal direction of the disk
radius = 0.5
height = 1.0
# Object sphere is defined in yt/data_objects/selection_objects/disk.py
disk = ds.disk(center, normal, radius, height)
pp = ds.plot_particles(
    "p_x", "p_y", "p_w", region=disk, unit_type="si", x_bins=64, y_bins=64
)
pp.show()

### Plot the velocity space of particles that are inside a disk

In [ ]:
pp = ds.plot_phase(
    "p_uy", "p_uz", "p_w", region=disk, unit_type="si", x_bins=64, y_bins=64
)

pp.show()

Get the 2D array from the phase plot and reconstruct from scratch:

In [ ]:
for var_name in pp.profile.field_data:
    val = pp.profile.field_data[var_name]

x = pp.profile.x
y = pp.profile.y

plt.pcolormesh(x, y, val)
plt.show()

# Analyzing AMReX Outputs From FLEKS (from demo_amrex_output.ipynb)

In this demo, we show how to analyze native AMReX field and particle outputs. Example data can be downloaded as follows:

In [ ]:
import flekspy
from flekspy.util import download_testfile

url = "https://raw.githubusercontent.com/henry2004y/batsrus_data/master/fleks_particle_small.tar.gz"
download_testfile(url, "data")

Inheriting from the IDL procedures, we can pass strings to limit the plotting range (experimental, may be changed in the future):

In [ ]:
ds = flekspy.load("data/fleks_particle_small/3d*amrex")
dc = ds.get_slice("z", 0.001)

f, axes = dc.plot("Bx>(2.2e5)<(3e5) Ex", figsize=(12, 6))
dc.add_stream(axes[0], "Bx", "By", color="w")
dc.add_contour(axes[1], "Bx", color="k")

## Velocity Space Distributions

In [ ]:
from flekspy.util import unit_one

filename = "data/fleks_particle_small/cut_*amrex"
ds = flekspy.load(filename)

# Add a user defined field. See yt document for more information about derived field.
ds.add_field(
    ds.pvar("unit_one"), function=unit_one, sampling_type="particle", units="code_mass"
)

x_field = "p_uy"
y_field = "p_uz"
z_field = "unit_one"
# z_field = "p_w"
xleft = [-0.001, -0.001, -0.001]
xright = [0.001, 0.001, 0.001]

### Select and plot the particles inside a box defined by xleft and xright
region = ds.box(xleft, xright)
pp = ds.plot_phase(
    x_field,
    y_field,
    z_field,
    region=region,
    unit_type="si",
    x_bins=64,
    y_bins=64,
    domain_size=(-0.0002, 0.0002, -0.0002, 0.0002),
)

pp.set_cmap(pp.fields[0], "turbo")

# plot.set_zlim(plot.fields[0], zmin, zmax)
pp.set_xlabel(r"$V_y$")
pp.set_ylabel(r"$V_z$")
# pp.set_colorbar_label(pp.fields[0], "pw")
pp.set_title(pp.fields[0], "Number density")
pp.set_font(
    {
        "size": 34,
        "family": "DejaVu Sans",
    }
)
pp.set_log(pp.fields[0], False)
pp.show()

If you need the direct phase space distributions together with the axis,

In [ ]:
x, y, w = ds.get_phase(
    x_field,
    y_field,
    z_field,
    region=region,
    x_bins=64,
    y_bins=64,
    domain_size=(-0.0002, 0.0002, -0.0002, 0.0002),
)

To check the newly added field,

In [ ]:
ad = ds.all_data()
ad[("particles", "unit_one")]

Plot the location of particles that are inside a sphere

In [ ]:
center = [0, 0, 0]
radius = 0.001
z_field = "unit_one"
# Object sphere is defined in yt/data_objects/selection_objects/spheroids.py
sp = ds.sphere(center, radius)
pp = ds.plot_particles(
    "p_x", "p_y", z_field, region=sp, unit_type="planet", x_bins=64, y_bins=64
)
pp.show()

Plot the phase space of particles that are inside a sphere

In [ ]:
pp = ds.plot_phase(
    "p_uy", "p_uz", z_field, region=sp, unit_type="planet", x_bins=64, y_bins=64
)
pp.show()

Plot the location of particles that are inside a disk

In [ ]:
center = [0, 0, 0]
normal = [1, 1, 0]
radius = 0.0005
height = 0.0004
z_field = "unit_one"
# Object sphere is defined in yt/data_objects/selection_objects/disk.py
disk = ds.disk(center, normal, radius, height)
pp = ds.plot_particles(
    "p_x", "p_y", z_field, region=disk, unit_type="planet", x_bins=64, y_bins=64
)
pp.show()

Plot the phase space of particles that are inside a disk

In [ ]:
pp = ds.plot_phase(
    "p_uy", "p_uz", z_field, region=disk, unit_type="planet", x_bins=64, y_bins=64
)
pp.show()

## Transform the velocity coordinates and visualize the phase space distribution

WIP

In [ ]:
import flekspy, yt

l = [1, 0, 0]
m = [0, 1, 0]
n = [0, 0, 1]


def _vel_l(field, data):
    res = (
        l[0] * data[("particles", "p_ux")]
        + l[1] * data[("particles", "p_uy")]
        + l[2] * data[("particles", "p_uz")]
    )
    return res


def _vel_m(field, data):
    res = (
        m[0] * data[("particles", "p_ux")]
        + m[1] * data[("particles", "p_uy")]
        + m[2] * data[("particles", "p_uz")]
    )
    return res


def _vel_n(field, data):
    res = (
        n[0] * data[("particles", "p_ux")]
        + n[1] * data[("particles", "p_uy")]
        + n[2] * data[("particles", "p_uz")]
    )
    return res


filename = "data/fleks_particle_small/cut_*amrex"
ds = flekspy.load(filename)

# Add a user defined field. See yt document for more information about derived field.
vl_name = ds.pvar("vel_l")
vm_name = ds.pvar("vel_m")
vn_name = ds.pvar("vel_n")
ds.add_field(vl_name, units="code_velocity", function=_vel_l, sampling_type="particle")
ds.add_field(vm_name, units="code_velocity", function=_vel_m, sampling_type="particle")
ds.add_field(vn_name, units="code_velocity", function=_vel_n, sampling_type="particle")

######## Plot the location of particles that are inside a sphere ###########
center = [0, 0, 0]
radius = 0.001
# Object sphere is defined in yt/data_objects/selection_objects/spheroids.py
sp = ds.sphere(center, radius)

x_field = vl_name
y_field = vm_name
z_field = ds.pvar("p_w")

logs = {x_field: False, y_field: False}
profile = yt.create_profile(
    data_source=sp,
    bin_fields=[x_field, y_field],
    fields=z_field,
    n_bins=[64, 64],
    weight_field=None,
    logs=logs,
)

pp = yt.PhasePlot.from_profile(profile)

pp.set_unit(x_field, "km/s")
pp.set_unit(y_field, "km/s")
pp.set_unit(z_field, "amu")

pp.set_cmap(pp.fields[0], "turbo")
# pp.set_zlim(pp.fields[0], zmin, zmax)
pp.set_xlabel(r"$V_l$")
pp.set_ylabel(r"$V_m$")
pp.set_colorbar_label(pp.fields[0], "colorbar_label")
pp.set_title(pp.fields[0], "Density")
pp.set_font(
    {
        "size": 34,
        "family": "DejaVu Sans",
    }
)
pp.set_log(pp.fields[0], False)
pp.show()